In [1]:
#全景画像(Target)と背景画像(Background)を合体させます
#Copyright (c) 2018  Shotaro Ishigami

In [2]:
import cv2
import numpy as np
from IPython.display import display, Image as iP
from PIL import Image, ImageDraw, ImageFilter
from ipywidgets import interact
import glob,sys
import random
import os
#画像表示用
import matplotlib.pyplot as plt
def display_cv_image(image, format='.png'):
    decoded_bytes = cv2.imencode(format, image)[1].tobytes()
    display(iP(data=decoded_bytes))

In [3]:
cv2.__version__

'3.4.4'

In [4]:
TARGET_DIR="./Target.bak/"
OUTDIR="./Output.bak/"


In [5]:
def Extraction_color(B,G,R,tar,dilate=0,erode=0):
    B_thres_type=cv2.THRESH_BINARY_INV
    B_thres=B
    G_thres_type=cv2.THRESH_BINARY_INV
    G_thres=G
    R_thres_type=cv2.THRESH_BINARY_INV
    R_thres=R
#     img=cv2.imread("Target/target1.jpg")
    BGR = cv2.split(tar) #BGRに分離
    _, B_mask = cv2.threshold(BGR[0], B_thres, 1, B_thres_type)
    _, G_mask = cv2.threshold(BGR[1], G_thres, 1, G_thres_type)
    _, R_mask = cv2.threshold(BGR[2], R_thres, 1, R_thres_type)
    background_mask = B_mask * G_mask * R_mask
    result = cv2.merge(BGR + [255 * background_mask])
    #display_cv_image(result,".png")
    result=Closing(result,dilate=dilate,erode=erode)
    return result


In [6]:
def PasteTarget(x=0,y=0,background=None,mask=None):
    target=Image.open("./tmp/tmp.png").convert("RGBA")
    print(target.mode)
    #マスク画像が指定された時(Extractionが呼ばれてる時)はmask処理をする
    if mask is None:
        background.paste(target,(x,y),target)
    else:
        mask_img=Image.open(mask)#.convert("RGBA")
        print(mask_img.mode)
        background.paste(target,(x,y),mask_img)
    return background

In [7]:
#画像を２値化します。
def Binarize(gray,im):
   
    im_gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)#画像の読み込み
    #ガウスフィルタ
    im_gray_smooth=cv2.GaussianBlur(im_gray,(11,11),0)
    #2値化
    _,th1 = cv2.threshold(im_gray_smooth,gray,255,cv2.THRESH_BINARY)
    
    return th1

In [8]:
#２値化した情報からマスクを作成します。
def MakeMask(th1,dilate=0,erode=0):
    #マスク画像作成
    mask_img=255-th1
    mask_img=Closing(mask_img,dilate=dilate,erode=erode)
    cv2.imwrite("./tmp/mask.png",mask_img)

In [9]:
# ターゲット画像を背景の２倍の大きさにします。
def img_expansion(img,resize,width=1900,height=1080):
    scale=2 #背景の何倍に拡張するか
    img.thumbnail((width/scale*resize,height/scale*resize))
    # 透明に塗りつぶす用の背景画像を作成
    bg = Image.new("RGBA",[width,height],(255,255,255,0))
    # 元の画像を、背景画像のセンターに配置
    bg.paste(img,(int((width-img.size[0])/2),int((height-img.size[1])/2)))
#     bg.paste(img,(0,0))
    return bg

In [10]:
#ターゲット切り取り
def ImageTrim(img):

    th1=Binarize(245,img)
    # 輪郭を抽出
      #   contours : [領域][Point No][0][x=0, y=1]
      #   cv2.CHAIN_APPROX_NONE: 中間点も保持する
      #   cv2.CHAIN_APPROX_SIMPLE: 中間点は保持しない
    _,contours, hierarchy = cv2.findContours(th1, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    
    h,w,_=img.shape[:3]
    img_area=h*w
    
    maxContour = 0
#     c=0###########################

    # 各輪郭に対する処理
    for i in range(0, len(contours)):

        # 輪郭の領域を計算
        area = cv2.contourArea(contours[i])

        # ノイズ（小さすぎる領域）と画像全体の輪郭を除外
        if area < 1e3 or area>=img_area-100000:
              continue

        # 外接矩形
        if len(contours[i]) > 0:
            if(area>maxContour):
                rect = contours[i]
                x, y, w, h = cv2.boundingRect(rect)
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0,0 ), 1)
#                 cv2.imwrite("./tmp/"+str(c)+"trim.png",img)##############
#                 c+=1####################
                # 外接矩形毎に画像を保存
                img_pe=img[y:y + h, x:x + w]
                maxContour=area
        
    print("img_area:"+str(img_area))
    print("area:"+str(area))
    try:
        return img_pe
    except NameError:
        return None

In [11]:
# 抽出モードによって抽出手法を変更します。maskを生成した場合、マスク画像のパスを返します。
def Extraction(Extraction_mode=None,
                        Threshold=None,
                        tar=None,
                        dilate=None,
                        erode=None,
                        B=None,G=None,R=None,
              ):
    
    if(Extraction_mode=="Binary"):
        #2値化による輪郭抽出
        th1=Binarize(Threshold,tar)
        MakeMask(th1,dilate,erode)
        mask=Image.open("./tmp/mask.png")
        tar=Image.fromarray(tar)
        tar.putalpha(mask)
        tar=np.asarray(tar)
       
    elif Extraction_mode=="Color":
        tar=Extraction_color(B,G,R,tar,dilate=dilate,erode=erode)#　色指定による輪郭抽出（BGR)
    #print(tar)
    return tar

    

In [12]:
#クロージングをします（拡大縮小率を変えられる）
def Closing(img,kernel=np.ones((5,5),np.uint8),dilate=0,erode=0):
    img_dilate=cv2.dilate(img,np.ones((5,5),np.uint8),iterations = dilate)
    img_dilate_erode = cv2.erode(img_dilate,np.ones((5,5),np.uint8),iterations = erode)
    return img_dilate_erode

In [13]:
# PILで明度調整
def PIL_point(point,img):

    img_dw=img.point(lambda x:x*point)

    return img_dw

In [14]:
#OpenCVで明度調整
def OpenCV_gamma(gamma,img):
    gamma_cvt=np.zeros((256,1),dtype='uint8')
    for i in range(256):
        gamma_cvt[i][0]=255*(float(i)/255)**(1.0/gamma)
    img_gamma=cv2.LUT(img,gamma_cvt)
    return img_gamma

In [15]:
#射影変換(ホモグラフィー変換)
def Homography(img,tar_w=None,tar_h=None,x1=0,y1=0,x2=0,y2=1080,x3=1900,y3=1080,x4=1900,y4=0):
    tar_h,tar_w,_=img.shape[:3]

    pts1 = np.float32([[0,0],[0,tar_h],[tar_w,tar_h],[tar_w,0]])#4隅を指定
    pts2 = np.float32([[x1,y1],[x2,y2+tar_h],[x3+tar_w,y3+tar_h],[x4+tar_w,y4]])#変換後
    M = cv2.getPerspectiveTransform(pts1,pts2)
    return cv2.warpPerspective(img,M,(tar_w,tar_h),borderValue=(255,255,255))

In [16]:
#ぼかし（ターゲット全体）
def TargetSmoothingBlur(img=None,Blur=None):
    if(Blur>0 and Blur%2!=0):
        dst=cv2.blur(img,ksize=(Blur,Blur))
        return dst
    else:
        print("ぼかしに使うカーネルは１以上で奇数の値を指定してください。")
        return img

In [17]:
#バイラテラルフィルタ
def BilateralBlur(img=None,cnt=1):
    if(cnt>1):
        dst=cv2.bilateralFilter(img,15,20,20)
        for i in range(cnt-1):
            dst=cv2.bilateralFilter(dst,15,20,20)
        return dst
    else:
        return img

In [18]:
# アフィン変換
def Affine(img,angle):
    h,w,c=img.shape
    M=cv2.getRotationMatrix2D(center=(w/2,h/2),angle=angle,scale=1.)
    dst=cv2.warpAffine(img,M,dsize=(w*2,h*2),borderValue=(255,255,255))
    return dst

In [19]:
# セピア変換
def Sepia(im):
    b, g, r = im[:,:,0],im[:,:,1], im[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b 
    r=gray*240/255
    g=gray*200/255
    b=gray*145/255
    im[:,:,0],im[:,:,1], im[:,:,2]=b, g, r
     
    return im

In [20]:
#鮮鋭化
def Sharp(img=None):
    kernel=np.array([[-1,-1,-1],
                     [-1,9,-1],
                     [-1,-1,-1]],np.float32)
    dst=cv2.filter2D(img,-1,kernel)
    return dst

In [21]:
#擬似遮蔽
#簡単な図形をランダムで生成し、遮蔽物を再現します。作成する個数は設定でき、初期値は０です。
#ターゲット周辺に生成されます。
def HideTarget(img,num):
    cv2.imwrite("./tmp/Shield.png",MakeShield(img,num))
    
    target=Image.open("./tmp/tmp.png").convert("RGBA")
    shield=Image.open("./tmp/Shield.png").convert("RGBA")
    target.paste(shield,(0,0),shield)
    return target

# 図形生成
def MakeShield(img,num=0):
    h,w,_=img.shape
    Shield=np.full((h,w,3),255,np.uint8)
    
    
    for i in range(num):
    
    #   描画設定
        ShieldType,FillOption,startX,startY,ColorR,ColorG,ColorB=Dise(w,h)

        if(ShieldType==0):#四角形    
            endX=random.randint(0,w)
            endY=random.randint(0,h)
            cv2.rectangle(Shield,(startX,startY),
                          (endX,endY),
                          (ColorB,ColorG,ColorR),FillOption)
        elif(ShieldType==1):#円
            min_len=min(h,w)
            radius=random.randint(1,min_len/2)
            cv2.circle(Shield,(startX,startY),
                       radius,(ColorB,ColorG,ColorR),FillOption)
        elif(ShieldType==2):#楕円
            radiusX=random.randint(1,w)
            radiusY=random.randint(1,h)
            angle=random.randint(1,359)
            cv2.ellipse(Shield,((startX,startY),
                                    (radiusX,radiusY),angle),
                                    (ColorB,ColorG,ColorR),FillOption)
    
    Shield=Extraction_color(B=254,G=254,R=254,tar=Shield)
    return Shield



def Dise(w,h):
    ShieldType=random.randint(0,2)
    FillOption=random.choice([-1,5])
    startX=random.randint(0,w)
    startY=random.randint(0,h)
    ColorR=random.randint(0,254)
    ColorG=random.randint(0,254)
    ColorB=random.randint(0,254)
    return ShieldType,FillOption,startX,startY,ColorR,ColorG,ColorB

In [22]:
# 画像反転
def flip_img(tar=None,flips=2):
    if not flips in [1,0,-1]:
        return tar
    img = cv2.flip(tar, flips)    
    return img

In [23]:
# ソルト&ペッパノイズ
def salt_and_pepper(tar=None, amount=0.,sp_ratio=0.5):
    #tar=np.asarray(tar)
    sp_img = tar.copy()
    #print(tar)
    # 塩モード
    num_salt = np.ceil(amount * tar.size * sp_ratio)
    coords = [np.random.randint(0, i-1 , int(num_salt)) for i in tar.shape]
    #print(coords[:-1])
    sp_img[coords[:-1]] = (255,255,255)

    # 胡椒モード
    num_pepper = np.ceil(amount* tar.size * (1. - sp_ratio))
    coords = [np.random.randint(0, i-1 , int(num_pepper)) for i in tar.shape]
    sp_img[coords[:-1]] = (0,0,0)
    
    return sp_img

In [24]:
# ガウシアンノイズ
def gauss_noise(tar=None,mean=0,sigma=15):
    row,col,ch= tar.shape
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    gauss_img = tar + gauss
    
    gauss_img=gauss_img.astype("int")
    gauss_img=np.asarray(gauss_img)
    return gauss_img

In [25]:
# 量子化クラスタリング
def cluster(tar=None,kernel=256):
    cluster_img = tar.reshape((-1,3))
    # convert to np.float32
    cluster_img = np.float32(cluster_img)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center=cv2.kmeans(cluster_img,kernel,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    result = center[label.flatten()]
    result = result.reshape((tar.shape))
    return result

In [31]:
def Nature_Target(tar=None,bak=None,x=0,y=0):
    tar_h,tar_w,_=tar.shape[:3]
    _,_,_,alpha=cv2.split(tar)
    bak=np.asarray(bak)
    bak=bak[y:y+tar_h,x:x+tar_w]
    bak = cv2.cvtColor(bak, cv2.COLOR_BGR2HSV)
    h_bak,s_bak,v_bak=cv2.split(bak)
  
    mean=int(np.mean(v_bak))
    
    
    tar=cv2.cvtColor(tar, cv2.COLOR_BGR2HSV)
    h_tar,s_tar,v_tar=cv2.split(tar)
    v_mean=np.full((tar_h,tar_w),mean,'uint8')
    v_mean=v_tar//2+v_mean//2
    
    tar=cv2.merge([h_tar,s_tar,v_mean])
    tar=cv2.cvtColor(tar, cv2.COLOR_HSV2BGR)
    tar=cv2.merge([tar,alpha])
    
    return tar

In [32]:
# 画像を合成します。
#すべての引数は省略できます。
#bak：背景画像
#tar：前景画像（ターゲット）
#SharpBackground：背景を鮮鋭化するか
#Brightness_mode：明るさを変更する方法を選択(No,PIL,OpenCV)
#dilate,erode：クロージング時の拡大率と縮小率
#X,Y：前景画像の貼り付け位置（途中の処理で前景画像をリサイズするのでその後の貼り付け位置）
#resize：前景画像を倍数指定でリサイズします
#R,G,B：切り抜かれる色を指定(0〜255）
#Threshold：２値化のの際の閾値（輪郭抽出）
#brightness：明るさ。
#BilateralFilter：バイラテラルフィルタ
#GausianFilter：ガウシアンフィルタ
#HideNum：遮蔽物の数
#x1_Ho,y1_Ho,x2_Ho,y2_Ho=,x3_Ho,y3_Ho,x4_Ho,y4_Ho：射影変換。
def Union(label_name="?",
         bak="/home/kagamiwomiru/datasets/haikeigazou/1-1.jpg",
         tar="/home/kagamiwomiru/datasets/hyousiki/326-6.jpg",
         SharpBackground=False,
         SepiaAll=False,
         Brightness_mode="No",
         Extraction_mode="No",
         dilate=5,erode=5,
         X=0,Y=0,
         Random_rotation=False,
         angle=0,
         resize=1,
         R=244,G=248,B=243,
         Threshold=200,
         brightness=1.0,
         BilateralFilter=1,
         GausianFilter=1,
         HideNum=0,
         x1_Ho=0,y1_Ho=0,x2_Ho=0,y2_Ho=0,x3_Ho=0,y3_Ho=0,x4_Ho=0,y4_Ho=0,
         flips=2,
         amount=0.,
         sp_ratio=0.5,
         mean=0,
         sigma=0,
         kernel=256,
         gray=0
         ):
    if(os.path.isdir("./tmp") is False):
        os.mkdir("./tmp/")
    if(SharpBackground is True):
        #背景画像の鮮鋭化
        cv2.imwrite("./tmp/background.jpg",Sharp(cv2.imread(bak)))
        background=Image.open("./tmp/background.jpg")
    else:
        background=Image.open(bak)
    
    target=Image.open(tar)
    w,h=background.size
    tar_w,tar_h=target.size
   

    #前景画像を拡張
    target=img_expansion(target,resize,w,h)
#     target.save("./tmp/tmp_ex.png")    
    #PILからOpenCVへ変換
    target = cv2.cvtColor(np.array(target),cv2.COLOR_BGR2RGB)   

    # 反転
    target=flip_img(tar=target,flips=flips)
    
    #バイラテラルぼかし
    target=BilateralBlur(img=target,cnt=BilateralFilter)
    cv2.imwrite("./tmp/tmp.png",target)

    # 前景抽出選択
    target=Extraction(tar=target,dilate=dilate, erode=erode,B=B,G=G,R=R,Threshold=Threshold, Extraction_mode=Extraction_mode)
    cv2.imwrite("./tmp/tmp.png",target)    
        
    #ホモグラフィー変換
    target=Homography(cv2.imread("./tmp/tmp.png",-1),tar_w=tar_w,tar_h=tar_h,
                          x1=x1_Ho,y1=y1_Ho,x2=x2_Ho,y2=y2_Ho,x3=x3_Ho,y3=y3_Ho,x4=x4_Ho,y4=y4_Ho)

    #アフィン回転
    if Random_rotation:
        target=Affine(target,random.randint(0,360))
    else:
        target=Affine(target,angle)

    #トリミング
    target=ImageTrim(target)
    cv2.imwrite("./tmp/tmp.png",target)
    try:
        tar_h,tar_w,_=target.shape[:3]
    except AttributeError:
        print("射影変換に失敗しました。現在の値を0に近づけるとうまくいくかもしれません。")
        sys.exit(1)
    
    target=HideTarget(target,HideNum)
    target.save("./tmp/tmp.png")
    
    
    #ターゲット全体にガウシアンぼかし
    target=TargetSmoothingBlur(cv2.imread("./tmp/tmp.png",-1),Blur=GausianFilter)

    cv2.imwrite("tmp/tmp.png",target)
    tar_h,tar_w,_=target.shape[:3]
   
    #リサイズ
    target=cv2.resize(target,(int(tar_w*resize),int(tar_h*resize)))
#     target=cv2.resize(target,resize,resize)
    
    cv2.imwrite("./tmp/tmp.png",target)
    tar_h,tar_w,_=target.shape[:3]

    #背景に溶け込ませる
    target=Nature_Target(tar=cv2.imread("./tmp/tmp.png",-1),bak=background,x=X,y=Y)
    cv2.imwrite("./tmp/tmp.png",target)
    
    #OpenCVで明度変換
    if (Brightness_mode == "OpenCV"): 
        target=OpenCV_gamma(brightness,cv2.imread("./tmp/tmp.png",-1))
        cv2.imwrite("./tmp/tmp.png",target)

    
    #PILで明度変換
    if (Brightness_mode == "PIL"):
        target=PIL_point(brightness,Image.open("./tmp/tmp.png"))
        target.save("./tmp/tmp.png")

    
    out_image=PasteTarget(x=X,y=Y,background=background)#画像の貼り付け
    out_image.save("./tmp/result.png")

    # 量子化クラスタリング
    if not kernel==-1:
        cluster_img=cluster(tar= cv2.imread("./tmp/result.png"),kernel=kernel)
        cv2.imwrite("./tmp/result.png",cluster_img)
    
    # ソルト&ペッパー
    sp_image=salt_and_pepper(tar= cv2.imread("./tmp/result.png"), amount=amount,sp_ratio=sp_ratio)
    
    # ガウシアンノイズ
    gauss_image=gauss_noise(tar=sp_image,mean=mean,sigma=sigma)
    cv2.imwrite("./tmp/result.png",gauss_image)
    
    if(SepiaAll is True):
        SepiaImage=Sepia(cv2.imread("./tmp/result.png"))
        cv2.imwrite("./tmp/result.png",SepiaImage)
    
        

    print(label_name+","+str(X)+","+str(Y)+","+str(X+tar_w)+","+str(Y+tar_h))

    out_image=Image.open("./tmp/result.png")
    return plt.imshow(out_image)
#     return out_image

# テスト用

様々な変数を調整して、最適なパラメータを探します。

## 色で抽出

 初期設定では白色を消すようにしているので、前景画像によっては消えすぎてしまうことがある。
 背景画像と前景画像を合成する関数 **Union**ではこの問題を回避するため、引数で消す色を指定できる。（RGB）



## 引数

- label_name:割り振るラベルIDを入力します。
- bak：背景画像
- tar：前景画像（ターゲット）
- SharpBackground：背景を鮮鋭化するか
- Extraction_mode：抽出方法を選択(輪郭抽出、色で抽出)
- Brightness_mode：明るさを変更する方法を選択(No,PIL,OpenCV)
- dilate,erode：クロージング時の拡大率と縮小率
- decrease：前景画像の縮小率(ターゲットが大きすぎるときに縮小率を整数で指定します）
- X,Y：前景画像の貼り付け位置（途中の処理で前景画像をリサイズするのでその後の貼り付け位置）
- R,G,B：切り抜かれる色を指定(0〜255）
- Threshold：２値化のの際の閾値（輪郭抽出）
- brightness：明るさ。(Brightness modeがNo以外で適用）
- BilateralFilter：バイラテラルフィルタ
- GausianFilter：ガウシアンフィルタ
- HideNum：遮蔽物の数
- x1_Ho,y1_Ho,x2_Ho,y2_Ho=,x3_Ho,y3_Ho,x4_Ho,y4_Ho：射影変換。
- flips : 反転(0:上下反転, 1:左右反転, -1:上下左右反転)
- amount : ソルト&ペッパにおける画像の中のノイズの割合
- sp_ratio : ソルト&ペッパの塩と胡椒の割合
- mean : ガウシアンフィルタにおける平均
- sigma : ガウシアンフィルタにおける分散
- kernel : 量子化するときのくくりの数値(つまりは何色で表示にするか)

In [33]:
interact(Union,img_expansion_mode=True,
         label_name="?",
         bak=glob.glob("./haikeigazou/*"),
         tar=glob.glob("./hyousiki/1/*"),
         SharpBackground=False,
         SepiaAll=False,
         Brightness_mode=["No","PIL","OpenCV"],
         Extraction_mode=["No","Color","Binary"],
         dilate=(0,5,1),erode=(0,5,1),
         Random_rotation=False,
         angle=(0,180,1),
         resize=(0.1,10,0.1),
         X=(0,2000,50),Y=(0,2000,50),
         R=(0,255,1),G=(0,255,1),B=(0,255,1),
         Threshold=(0,255,1),
         brightness=(0.1,2.0,0.1),
         BilateralFilter=(1,10,1),
         GausianFilter=(1,9,2),
         HideNum=(0,10,1),
         x1_Ho=(-1500,1500,5),y1_Ho=(-1500,1500,5),x2_Ho=(-1500,1500,5),y2_Ho=(-1500,1500,5),x3_Ho=(-1500,1500,5),y3_Ho=(-1500,1500,5),x4_Ho=(-1500,1500,5),y4_Ho=(-1500,1500,5),
         flips=(-1,2,1),
         amount=(0.,0.05,0.001),
         sp_ratio=(0.,1.,0.1),
         mean=(0,100,1),
         sigma=(0,100,1),
         kernel=(-1,16,1),
         gray=(0,255,1)
         )

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHQodmFsdWU9dSc/JywgZGVzY3JpcHRpb249dSdsYWJlbF9uYW1lJyksIERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnYmFrJywgb3B0aW9ucz0oJy4vaGHigKY=


<function __main__.Union>